# Use LangChain with AI21 Jamba 1.5 Large and Jamba 1.5 Mini for Azure AI Models-as-a-Service

You can use AI21 Jamba 1.5 Large and Jamba 1.5 Mini deployed as a serverless API endpoint through Azure AI Studio and Azure ML with `langchain` to create more sophisticated intelligent applications.

> Review the [documentation](https://aka.ms/ai21-jamba-1.5-large-azure-ai-studio-docs) for AI21 Jamba 1.5 Large through AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `AI21 Jamba 1.5 Large` or `AI21 Jamba 1.5 Mini`

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://aka.ms/ai21-jamba-1.5-large-azure-ai-studio-docs) for model deployment and inference.

To complete this tutorial, you will need to:

* Install `langchain` and `langchain_ai21`:

    ```bash
    pip install langchain langchain-ai21
    ```

## Example

The following example demonstrate how to create a chain that uses AI21 Jamba 1.5 Large model deployed through Azure AI and Azure ML. The chain has been configured with a `ConversationBufferMemory`. This example has been adapted from [LangChain official documentation](https://python.langchain.com/docs/modules/memory/adding_memory).

In [ ]:
%pip install langchain
%pip install langchain-ai21
%pip install -U "ai21>=2.13.0"

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_ai21.chat_models import ChatAI21

In the below prompt, we have two input keys: one for the actual input (`human_input`), and another for the input from the `Memory` class (`chat_history`).

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. You love making references to pop culture in  your answers."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

Let's create an instance of our model deployed in Azure AI or Azure ML. Use `langchain_ai21` package and configure it as follows:

- `endpoint`: Use the endpoint URL from your deployment. Include `/v1` at the end of the endpoint URL.
- `api_key`: Use your API key.

In [ ]:
chat_model = ChatAI21(api_host="<your-maas-endpoint>", api_key="<your-api-key>")

> Tip: Alternatively, you can configure your API key in the environment variables `AI21_API_KEY`.

We create the chain as follows:

In [ ]:
chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

We can see how it works:

In [ ]:
chat_llm_chain.predict(human_input="Hi there my friend")

In [ ]:
chat_llm_chain.predict(
    human_input="Who is the most renowned architect in the world? Provide a short poem that describes their work in the style of Shakespeare with Iambic pentimeter and a rhythm pattern of ABABCC"
)

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)